!pip install selenium

In [27]:
import requests
import json

In [28]:
def get_twentyninecm_category(category1_code):
    url = 'https://cache.29cm.co.kr/item/category/'
    params = {
        'category1_code': category1_code,
        'sortBy': 'VISIT_COUNT'
    }
    response = requests.get(url, params=params)

    try:
        response.raise_for_status()
        print("Request was successful!")
    except requests.exceptions.HTTPError as err:
        print(f"HTTP Error: {err}")

    results = response.json()
    return results


get_twentyninecm_category(268100100)

Request was successful!


{'count': 1,
 'next': None,
 'previous': None,
 'results': [{'category': {'category_name': '여성의류',
    'category_code': 268100100,
    'category2': [{'category_name': '상의',
      'category_code': 268103100,
      'category3': [{'category_name': '긴소매 티셔츠',
        'category_code': 268103102,
        'count': 0},
       {'category_name': '스웨트셔츠', 'category_code': 268103104, 'count': 0},
       {'category_name': '블라우스', 'category_code': 268103107, 'count': 0},
       {'category_name': '셔츠', 'category_code': 268103106, 'count': 0},
       {'category_name': '반소매 티셔츠', 'category_code': 268103101, 'count': 0},
       {'category_name': '후디', 'category_code': 268103105, 'count': 0},
       {'category_name': '슬리브리스', 'category_code': 268103103, 'count': 0}],
      'count': 0},
     {'category_name': '아우터',
      'category_code': 268102100,
      'category3': [{'category_name': '재킷',
        'category_code': 268102104,
        'count': 0},
       {'category_name': '점퍼', 'category_code': 268102119

In [29]:
def get_twentyninecm_products_list(categoryLargeCode, categoryMediumCode, categorySmallCode):
    url = 'https://search-api.29cm.co.kr/api/v4/products/category/'
    params = {
        'categoryLargeCode': categoryLargeCode,
        'categoryMediumCode': categoryMediumCode,
        'categorySmallCode': categorySmallCode,
        'count': 2,
        'page': 1,
        'sort': 'latest',
        'init': 'T',
        'excludeSoldOut': False,
    }

    # 카테고리별로 상품 2개만 조회 -> res의 productsTotalCount를 저장 -> 모든 상품 조회
    is_get_count = False
    while True:
        response = requests.get(url, params=params)
        
        try:
            response.raise_for_status()
            print("Request was successful!")
        except requests.exceptions.HTTPError as err:
            print(f"HTTP Error: {err}")
    
        results = response.json()

        if is_get_count:
            return results
            
        params['count'] = results.get('data', {}).get('productsTotalCount', 0)
        is_get_count = True


get_twentyninecm_products_list(268100100, 268103100, 268103102)

Request was successful!
Request was successful!


{'result': 'SUCCESS',
 'data': {'products': [{'itemNo': 2469332,
    'itemName': 'NU NOSTALGIA LONG SLEEVE',
    'frontBrandNo': 16723,
    'frontBrandNameKor': '글로니',
    'frontBrandNameEng': 'GLOWNY',
    'categoryNames': None,
    'consumerPrice': 72000,
    'sellPrice': 64800,
    'imageUrl': '/item/202403/11eedba99778179291a5efbae40926c1.jpg',
    'visibleBeginTimestamp': '2024-03-11T09:40:00.024000+09:00',
    'visibleEndTimestamp': None,
    'useWithoutVisibleEndDate': 'T',
    'availableBeginTimestamp': '2024-03-06T20:12:20.450217+09:00',
    'availableEndTimestamp': None,
    'useWithoutAvailableEndDate': None,
    'discountRate': 10,
    'couponDiscountRate': 5,
    'isCouponAllowedInOrder': True,
    'heartCount': 1041,
    'reviewCount': 0,
    'reviewAveragePoint': 0.0,
    'isSoldOut': False,
    'lastSalePrice': 61560,
    'lastSalePercent': 15,
    'subjectDescriptions': [],
    'isFreeShipping': True,
    'isNew': True,
    'heartOn': False,
    'saleInfoV2': {'consume

- product
    - main_category
    - sub_category
    - gender: 
    - name: itemName
    - price: consumerPrice
    - quantity: 크롤링
    - brand_name: frontBrandNameKor, frontBrandNameEng
    - size: 크롤링
    - color: 크롤링
    - fee: 
    - image: https://img.29cm.co.kr/{imageUrl}
    - code: 크롤링
    - url: https://product.29cm.co.kr/catalog/{item_no}

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

def crawling_twentyninecm_product_info(itemNo):
    product_info = {'size': [], 'color': [], 'code': ''}
    product_images = []
    
    url = f'https://product.29cm.co.kr/catalog/{itemNo}'

    # Chrome 옵션 설정
    chrome_options = Options()
    # chrome_options.add_argument('--headless')  # 무인 모드로 실행
    chrome_options.add_experimental_option("detach", True)

    # WebDriver 실행
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)

    # 색상 및 사이즈 입력 요소 찾기
    color_input = driver.find_elements_by_xpath("//input[@placeholder='COLOR']")
    size_input = driver.find_elements_by_xpath("//input[@placeholder='SIZE']")

    # 존재하는 입력 요소가 있다면 값을 입력
    if color_input:
        color_input_value = 'Your Color Value'
        color_input[0].send_keys(color_input_value)

    if size_input:
        size_input_value = 'Your Size Value'
        size_input[0].send_keys(size_input_value)

        # dropbox가 생성된 뒤에 ul 태그의 요소들을 추출함
        
    
    # 상품 조건 dropbox가 없는 경우 단일 사이즈/색상이라는 의미
    else:
        pass


crawling_twentyninecm_product_info(1896123)

AttributeError: 'WebDriver' object has no attribute 'find_elements_by_xpath'

In [9]:
def get_twentyninecm_reviews_list(itemNo):
    url = 'https://review-api.29cm.co.kr/api/v4/reviews/'
    params = {
        'itemId': itemNo,
        'page': 0,
        'size': 20,
    }
    response = requests.get(url, params=params)

    try:
        response.raise_for_status()
        print("Request was successful!")
    except requests.exceptions.HTTPError as err:
        print(f"HTTP Error: {err}")

    results = response.json()
    return results


get_twentyninecm_reviews_list(1896123)

Request was successful!


{'result': 'SUCCESS',
 'data': {'count': 349,
  'giftCount': 0,
  'averagePoint': 4.5,
  'results': [{'itemReviewNo': 8252510,
    'optionValue': ['[SIZE]FREE'],
    'userNo': 7393594,
    'userId': 'jds10***',
    'point': 4,
    'contents': '도톰한길이로.이너웨어로입기좋아요.목도적당하게파여답답하지않고팔길이도길어서겨울에손등까지덮기좋네요.167기준.길이는크롭인거같아요.밑위긴하의입으면배는가릴수있어요',
    'reviewType': 0,
    'registrationType': 'USER',
    'isGift': 'F',
    'isReported': 'F',
    'giftReview': None,
    'userSize': ['165cm', '53kg'],
    'uploadFiles': [],
    'partnerComment': None,
    'surveyList': [{'surveyType': 'SIZE', 'optionValue': 2}],
    'insertTimestamp': '2024-03-14 07:24:37',
    'orderNo': None,
    'orderSerial': None,
    'adminId': None,
    'adminName': None},
   {'itemReviewNo': 8251821,
    'optionValue': ['[SIZE]FREE'],
    'userNo': 4603237,
    'userId': 'pretty6***',
    'point': 5,
    'contents': '핑크 포인트!! 보자마자 반했어요ㅠ\n레이어드용으로 이너로 입기 좋고 팔이 길어서 좋아요\n포인트로 좋습니다',
    'reviewType': 0,
    'registrationType': 'USER',
